In [ ]:
# Importando as bibliotecas:

from PIL import Image
from os import listdir
from os.path import isdir
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers

In [ ]:
# Carregando uma imagem:

def select_image(filename):
    image = image.convert('RGB')
    image = image.resize((150,150))
    
    return np.asarray(image)

In [ ]:
# Carregando uma classse:

def load_Class(diretorio, classe, imagens, labels):
    # Iterando arquivos
    
    for filename in listdir(diretorio):
        
        path = diretorio + filename
        
        try:
            imagens.append(select_image(path))
            labels.append(classe)
        except:
            print("Erro ao ler imagem {}".format(path))

In [ ]:
# Selecionado o Dataset:

def select_data_set(diretorio):
    
    imagens = list()
    labels = list()
    
    for subdir in listdir(diretorio):
        # path
        path = diretorio + subdir + '//'
        
        if not isdir(path):
            continue
        imagens, labels = load_Class(path, subdir, imagens, labels)
        
    return imagens, labels

In [ ]:
# Carregando o dataset COVID-19:

covid_dataset = "//home//jovyan//dataset//imagens//"
imagens, labels = select_data_set(covid_dataset)

In [ ]:
imagens = np.array(imagens) / 255.0
imagens = np.array(labels)

In [ ]:
imagens

In [ ]:
# Tratando classes - Labels:

lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [ ]:
# Definindo hyperparemetros:

batch_size = 32 
input_shape = (150, 150, 3) 
radom_state = 42 
alpha = 1e-5  
epoch = 100 

In [ ]:
from tensorflow.kerascallbacks import ModelCheckpoint, ReduceLROnPlateu

In [ ]:
# usamos um arquivo para salvar:

filepath="transferlearning_weights.hdf5"
chepoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max_val_acc')

In [ ]:
ir_reduce = ReduceLROnPlateau(monitor='val_acc', factor=0.1, min_delta=alpha, patience5, verbose=1)

In [ ]:
# Array de calbacks:

callbacks = [checkpoint, lr_reduce]

In [ ]:
# Particionando dataset em teste e treino:

(trainX, testeX, trainY, testY) = train_test_split(imagens, labels, teste_size=0.2, stratify=labels, radom_state=radom_state)

In [ ]:
train_datagen = ImageDataGenerator(rotation_range=20, zoom_range=0.2)

train_datagen.fit(trainX) 

data_aug = train_datagen.flow(trainX, trainY, batch_size=batch_size) 

In [ ]:
# Transfer Learning:

conv_base = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)

In [ ]:
conv_base.summary()

In [ ]:
#vou dizer que a rede é toda treinavel:

conv_base.treinable = True
set_trainable = False // falso

for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
conv_base.sumary()

In [ ]:
# Criando o modelo com o VGG19 como base:

model = models.Sequencial()
model.add(conv_base) 
model.add(layers.GlobalAveragePooling2D()) 
model.add(layers.BatchNormalization()) 
model.add(layers.Flatten()) 
model.add(layers.Dense(128, activation='relu')) 
model.add(layers.Droupout(0.6)) 
model.add(layers.Dense(2, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
# Compilando o meu modelo:

model.compile(loss='binary_crossentropy', 
              optimizer='adam',
              metrics=['acc'])

In [ ]:
# Treinando o modelo - OBS: "Não faça isso agora, esse passo deve ser feito antes da apresetação:"
history = model.fit_generator(data_aug, 
                               steps_per_epoch=len(train) // batch_size,
                               validation_dat=(testX, testY),
                               validation_steps=len(testX) // batch_size,
                               callbacks=callbacks,
                               epochs=epoch)

In [ ]:
# Analisando os dados :

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
pltylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss:

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train, 'test'], loc='upper left'])
plt.show()

In [ ]:
# Gerando matriz de confusão:

from sklearn.metrics import confusion_matrix
pred = model.predict(testX)
pred = np.argmax(testY,axis =1)

In [ ]:
cm = confusion_matrix(y_true, pred)
total = sum(sum(cm))
acc = (cm[0,0] + cm[1, 1] / total
sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
       
print("Acurácia: {:.4f}".format(acc))
print("Sensitividade: {:.4f".format(sensitivity))
print("Especificidade: {:.4f}".format(specificity))